### 1. Import Packages

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
from pandas_datareader import data
import datetime

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from scipy.optimize import minimize

### 2. Setting Tickers and Functions

In [ ]:
# Market Capitalization Top-Ranked Companies in U.S.
tickers = ['AAPL', 'ADBE', 'AMZN', 'BAC', 'CMCSA', 'CSCO', 'CVX', 'DIS',
           'GOOG', 'HD', 'INTC', 'JNJ', 'JPM', 'KO', 'MA', 'MSFT', 'NFLX', 'NKE',
           'NVDA', 'ORCL', 'PFE', 'PG', 'T', 'TM', 'TSLA', 'UNH', 'V', 'VZ', 'WMT',
           'XOM']

In [ ]:
# Setting Device - If you have gpu, you can change the device 'gpu'
device = torch.device('cpu')

In [ ]:
# Make Class of LSTM Model
class LSTM_model(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM_model, self).__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.seq_length = seq_length #sequence length
 
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #LSTM layer
    self.fc = nn.Linear(hidden_size, num_classes) #fully connected last layer

  def forward(self,x):
    h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
    c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state   
    # Propagate input through LSTM

    out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
    # Decode the hidden state of the last time step
    out = self.fc(out[:, -1, :])
    return out

In [ ]:
def MAE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred)))

def MSE(y_true, y_pred):
    return np.mean(np.square((y_true - y_pred)))

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred))))

def MAPE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def MPE(y_true, y_pred): 
    return np.mean((y_true - y_pred) / y_true) * 100

### 3. Setting Hyperparameters and Training Data

In [ ]:
num_epochs = 10000
learning_rate = 0.001

input_size = 6 # number of features
hidden_size = 2 # number of features in hidden state
num_layers = 1 # number of stacked LSTM layers

num_classes = 1 #number of output classes 

In [ ]:
df_results_lstm = pd.DataFrame()
df_metrics_lstm = pd.DataFrame()

In [ ]:
for i in range(len(tickers)):
    df = pd.read_csv('./stock_price_data/' + tickers[i]+'.csv', index_col = 0)
    
    df_s = pd.DataFrame()
    
    df_s['High'] = (df['High'] - df['High'].min())/(df['High'].max() - df['High'].min())
    df_s['Low'] = (df['Low'] - df['Low'].min())/(df['Low'].max() - df['Low'].min())
    df_s['Open'] = (df['Open'] - df['Open'].min())/(df['Open'].max() - df['Open'].min())
    df_s['Close'] = (df['Close'] - df['Close'].min())/(df['Close'].max() - df['Close'].min())
    df_s['Volume'] = (df['Volume'] - df['Volume'].min())/(df['Volume'].max() - df['Volume'].min())
    df_s['Adj Close'] = (df['Adj Close'] - df['Adj Close'].min())/(df['Adj Close'].max() - df['Adj Close'].min())
    
    X = np.array(df_s) # Using high price, low price, Open price, Close price, Volume, and Adjusted Close Price to predict adjusted close price
    y = np.array(df_s.iloc[:, 5:6]) #Predicting adjusted close price

    train_pct = 0.8
    split = int(train_pct * len(X))
    X_train, X_test, y_train, y_test = X[:split, :], X[split+1:-1 :], y[1:split+1, :], y[split+2:, :] 

    X_train_tensors = Variable(torch.Tensor(X_train))
    X_test_tensors = Variable(torch.Tensor(X_test))

    y_train_tensors = Variable(torch.Tensor(y_train))
    y_test_tensors = Variable(torch.Tensor(y_test))

    X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
    X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

    LSTM_1 = LSTM_model(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to(device)

    loss_function = torch.nn.MSELoss()    # mean-squared error for regression
    optimizer = torch.optim.Adam(LSTM_1.parameters(), lr=learning_rate)  # adam optimizer

    print(tickers[i])
    for epoch in range(num_epochs+1):
        outputs = LSTM_1.forward(X_train_tensors_final.to(device)) #forward pass
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        loss = loss_function(outputs, y_train_tensors.to(device))

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop

        if epoch % 1000 == 0:
            print("Epoch: %d, Loss: %1.4f" % (epoch, loss.item()))

    X_data = X_test
    y_data = y_test

    df_X = Variable(torch.Tensor(X_data))
    df_y = Variable(torch.Tensor(y_data))

    df_X = torch.reshape(df_X, (df_X.shape[0], 1, df_X.shape[1]))
    train_predict = LSTM_1(df_X.to(device))#forward pass
    y_predict = train_predict.data.detach().cpu().numpy() #numpy conversion
    
    y_predict = y_predict * (df['Adj Close'].max() - df['Adj Close'].min()) + df['Adj Close'].min() #reverse transformation
    y_true = np.array(df.iloc[split+2:, 5:6])
    
    metrics_value_lstm = pd.DataFrame([MAE(y_true, y_predict), MSE(y_true, y_predict), RMSE(y_true, y_predict), MAPE(y_true, y_predict), MPE(y_true, y_predict)])
    metrics_value_lstm.columns = [tickers[i]]
    metrics_value_lstm.index = ['MAE', 'MSE', 'RMSE', 'MAPE', 'MPE']
    df_metrics_lstm = pd.concat([df_metrics_lstm, metrics_value_lstm], axis = 1)
    df_results_lstm_temp = pd.DataFrame(y_predict)
    df_results_lstm_temp.columns = [tickers[i]]
    df_results_lstm = pd.concat([df_results_lstm, df_results_lstm_temp], axis=1)

### 4-1. Saving Metrics

In [ ]:
df_metrics_lstm.to_csv('Results of Error Metrics (LSTM).csv', encoding = 'utf-8-sig')

### 4-2. Building Datasets

In [ ]:
df_results_lstm.index = df.iloc[len(df) - len(df_results_lstm):, ].index

In [ ]:
df_original = pd.DataFrame()

for i in range(len(tickers)):
    df = pd.read_csv('./stock_price_data/' + tickers[i]+'.csv', index_col = 0)
    close = df['Adj Close']
    df_close = pd.DataFrame(close)
    df_close.columns = [tickers[i]]
    df_original = pd.concat([df_original, df_close], axis=1)

ret_daily = df_results_lstm.pct_change()
ret_daily = ret_daily.iloc[1:, ]

ret_daily_original = df_original.iloc[len(df) - len(df_results_lstm) - 251:, ]
ret_daily_original = ret_daily_original.pct_change()
ret_daily_original = ret_daily_original.iloc[1:, ]

n_assets = len(tickers)

In [ ]:
df_original = pd.DataFrame()

for i in range(len(tickers)):
    df = pd.read_csv('./stock_price_data/' + tickers[i]+'.csv', index_col = 0)
    close = df['Adj Close']
    df_close = pd.DataFrame(close)
    df_close.columns = [tickers[i]]
    df_original = pd.concat([df_original, df_close], axis=1)

### 5-1. Portfolio Optimization with Stock Price Prediction Results

In [ ]:
def obj_sharpe(weights, returns, covmat, rf):
    ret = np.dot(weights, returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(covmat, weights)))
    return 1/((ret-rf)/np.sqrt(vol)) # 1/Sharpe Ratio -> Maximize Sharpe Ratio Using Minimize Function

In [ ]:
portfolio_lstm = []
for i in range(0, len(ret_daily)-1):
    ret_annual = pd.concat([ret_daily_original[i:i+251], ret_daily[i:i+1]]).mean() * 252
    cov_annual = pd.concat([ret_daily_original[i:i+251], ret_daily[i:i+1]]).cov() * 252
    rf = 0.01
    w0 = np.ones([n_assets])/n_assets
    bnds = tuple((0., 1.) for i in range(n_assets))
    cons = ({'type': 'eq', 'fun': lambda w:  np.sum(w) - 1}) 
    res = minimize(obj_sharpe, w0, (ret_annual, cov_annual, rf), method='SLSQP', bounds=bnds, constraints=cons)
    profit = (ret_daily_original.iloc[i+252] * res.x).sum()
    portfolio_lstm.append(profit)
    print(profit)

In [ ]:
portfolio_lstm = pd.DataFrame(portfolio_lstm) - 0.0003 # Upper bound of security transaction tax
portfolio_lstm.index = ret_daily.index[1:]
portfolio_lstm.columns = ['Portfolio Return']

data_lstm = pd.DataFrame([['2018-10-24',0]], columns=['Date','Portfolio Return'])
data_lstm.index = data_lstm['Date']
data_lstm = data_lstm.drop(['Date'], axis=1)

portfolio_lstm = pd.concat([data_lstm, portfolio_lstm])

In [ ]:
c_return_lstm = []

for i in range(0, len(portfolio_lstm)):
    if i == 0:
        c_return_lstm_v_init = 0
        c_return_lstm.append(c_return_lstm_v_init)
    else:
        c_return_lstm_v = (c_return_lstm[i-1] + 1) * (portfolio_lstm['Portfolio Return'][i] + 1) - 1
        c_return_lstm.append(c_return_lstm_v)
        
c_return_lstm = pd.DataFrame(c_return_lstm)
c_return_lstm.index = portfolio_lstm.index

portfolio_lstm_final = pd.concat([portfolio_lstm, c_return_lstm], axis = 1)
portfolio_lstm_final.columns = ['Portfolio Return', 'Cumulative Return']

In [ ]:
risk_free_rate = 0.015
std_portfolio_lstm = portfolio_lstm_final['Portfolio Return'][1:].std()
Sharpe_lstm = (portfolio_lstm_final['Portfolio Return'][1:] - risk_free_rate)/std_portfolio_lstm
Sharpe_lstm = Sharpe_lstm.mean()
print('Sharpe ratio (Portfolio with LSTM Price Prediction Results): %.2f' % Sharpe_lstm)

In [ ]:
portfolio_lstm_final.tail()

### 5-2. Equally-Weighted Portfolio

In [ ]:
portfolio_eqw = []

for i in range(0, len(ret_daily)-1):
    weights = np.ones([n_assets])/n_assets
    profit_eqw = (ret_daily_original.iloc[i+252] * weights).sum()
    portfolio_eqw.append(profit_eqw)

portfolio_eqw = pd.DataFrame(portfolio_eqw) 
portfolio_eqw.index = ret_daily.index[1:]
portfolio_eqw.columns = ['Portfolio Return']
    
data_eqw = pd.DataFrame([['2018-10-24',0]], columns=['Date','Portfolio Return'])
data_eqw.index = data_eqw['Date']
data_eqw = data_eqw.drop(['Date'], axis=1)

portfolio_eqw = pd.concat([data_eqw, portfolio_eqw])

c_return_eqw = []

for i in range(0, len(portfolio_eqw)):
    if i == 0:
        c_return_eqw_v_init = 0
        c_return_eqw.append(c_return_eqw_v_init)
    else:
        c_return_eqw_v = (c_return_eqw[i-1] + 1) * (portfolio_eqw['Portfolio Return'][i] + 1) - 1
        c_return_eqw.append(c_return_eqw_v)
        
c_return_eqw = pd.DataFrame(c_return_eqw)
c_return_eqw.index = portfolio_eqw.index

portfolio_eqw_final = pd.concat([portfolio_eqw, c_return_eqw], axis = 1)
portfolio_eqw_final.columns = ['Portfolio Return', 'Cumulative Return']

In [ ]:
risk_free_rate = 0.015
std_portfolio_eqw = portfolio_eqw_final['Portfolio Return'][1:].std()
Sharpe_eqw = (portfolio_eqw_final['Portfolio Return'][1:] - risk_free_rate)/std_portfolio_eqw
Sharpe_eqw = Sharpe_eqw.mean()
print('Sharpe ratio (Equally-Weighted Portfolio): %.2f' % Sharpe_eqw)

In [ ]:
portfolio_eqw_final.tail()

### 5-3. Capitalization Weighted Portfolio

In [ ]:
market_cap_data = data.get_quote_yahoo(tickers)['marketCap']
market_cap = pd.DataFrame(market_cap_data)

In [ ]:
market_cap = market_cap / market_cap.sum()

In [ ]:
market_cap = np.array(market_cap['marketCap'].tolist())

In [ ]:
portfolio_mcw = []

for i in range(0, len(ret_daily)-1):
    weights = market_cap
    profit_mcw = (ret_daily_original.iloc[i+252] * weights).sum()
    portfolio_mcw.append(profit_mcw)

portfolio_mcw = pd.DataFrame(portfolio_mcw) 
portfolio_mcw.index = ret_daily.index[1:]
portfolio_mcw.columns = ['Portfolio Return']

data_mcw = pd.DataFrame([['2018-10-24',0]], columns=['Date','Portfolio Return'])
data_mcw.index = data_mcw['Date']
data_mcw = data_mcw.drop(['Date'], axis=1)

portfolio_mcw = pd.concat([data_mcw, portfolio_mcw])

c_return_mcw = []

for i in range(0, len(portfolio_mcw)):
    if i == 0:
        c_return_mcw_v_init = 0
        c_return_mcw.append(c_return_mcw_v_init)
    else:
        c_return_mcw_v = (c_return_mcw[i-1] + 1) * (portfolio_mcw['Portfolio Return'][i] + 1) - 1
        c_return_mcw.append(c_return_mcw_v)
        
c_return_mcw = pd.DataFrame(c_return_mcw)
c_return_mcw.index = portfolio_mcw.index

portfolio_mcw_final = pd.concat([portfolio_mcw, c_return_mcw], axis = 1)
portfolio_mcw_final.columns = ['Portfolio Return', 'Cumulative Return']

In [ ]:
risk_free_rate = 0.015
std_portfolio_mcw = portfolio_mcw_final['Portfolio Return'][1:].std()
Sharpe_mcw = (portfolio_mcw_final['Portfolio Return'][1:] - risk_free_rate)/std_portfolio_mcw
Sharpe_mcw = Sharpe_mcw.mean()
print('Sharpe ratio (Market Capitalization-Weighted Portfolio): %.2f' % Sharpe_mcw)

In [ ]:
portfolio_mcw_final.tail()

### 6. Plotting Cumulative Returns of Portfolios

In [ ]:
plt.rcParams["figure.figsize"] = (32,18)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.color'] = 'b'
plt.rcParams['axes.grid'] = True 
plt.rcParams.update({'font.size': 50})
plt.rcParams['lines.linewidth'] = 3

In [ ]:
portfolio_lstm_final.index = pd.to_datetime(portfolio_lstm_final.index)
portfolio_eqw_final.index = pd.to_datetime(portfolio_eqw_final.index)
portfolio_mcw_final.index = pd.to_datetime(portfolio_mcw_final.index)

In [ ]:
line1 = plt.plot(portfolio_lstm_final.index.to_pydatetime(), portfolio_lstm_final['Cumulative Return'], label = 'Portfolio Using LSTM Prediction Results', color = 'red')
line2 = plt.plot(portfolio_eqw_final.index.to_pydatetime(), portfolio_eqw_final['Cumulative Return'], label = 'Equally Weighted Portfolio', color = 'deepskyblue')
line3 = plt.plot(portfolio_mcw_final.index.to_pydatetime(), portfolio_mcw_final['Cumulative Return'], label = 'Capitalization Weighted Portfolio', color = 'blue')
plt.xticks(fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.ylim(-0.6, 5.1)
plt.legend(prop={'size':15})
plt.show()